# Using FRED API to Obatin Economic Data

### Modules and Installations

In [1]:
from dotenv import load_dotenv
import os
from fredapi import Fred
import requests
import pandas as pd

In [2]:
# Only use if the 'dotenv' module is not installed
!pip install python-dotenv

In [3]:
# Only use if the 'fredapi' module is not installed
!pip install fredapi

### Create Fred Object and Use the API Key

In [4]:
load_dotenv()

True

In [5]:
api_key = os.environ.get('FRED_API_KEY')

In [6]:
fred = Fred(api_key=api_key)

### Gather the Data of five Economic Indicators

In [7]:
# Federal Funds Effective Rate, Monthly frequency

ff = fred.get_series('ff', frequency='m')

In [8]:
ff_df = pd.DataFrame(ff, columns=['FF'])

In [9]:
ff_df.head()

,FF
1954-07-01,0.86
1954-08-01,1.10
1954-09-01,1.10
1954-10-01,0.84
1954-11-01,0.74


In [10]:
# Unemployment Rate, Monthly frequency

u3 = fred.get_series('UNRATE', frequency='m')

In [11]:
u3_df = pd.DataFrame(u3, columns=['U-3'])

In [12]:
u3_df.head()

,U-3
1948-01-01,3.4
1948-02-01,3.8
1948-03-01,4.0
1948-04-01,3.9
1948-05-01,3.5


In [13]:
# Consumer Price Index for All Urban Consumers: All Items in U.S. City Average, Monthly frequency

cpi = fred.get_series('CPIAUCSL', frequency='m')

In [14]:
cpi_df = pd.DataFrame(cpi, columns=['CPI'])

In [15]:
# We need to do some data calculation in order to get the year-over-year change:

cpi_df['CPI_%_Change'] = cpi_df['CPI'].pct_change(periods=12).mul(100).round(2)

In [16]:
cpi_df.sample(5, axis = 0)

,CPI,CPI_%_Change
1952-06-01,26.530,2.31
2013-03-01,232.282,1.52
2001-05-01,177.300,3.56
2007-08-01,207.667,1.90
1972-04-01,41.500,3.49


In [17]:
# M2 (money supply - includes highly liquid assets), Monthly frequency
m2 = fred.get_series('M2SL', frequency='m')

In [18]:
m2_df = pd.DataFrame(m2, columns=['M2'])

In [20]:
# We need to do some data calculation in order to get the year-over-year change:

m2_df['M2_%_Change'] = m2_df['M2'].pct_change(periods=12).mul(100).round(2)

In [21]:
m2_df.sample(5, axis = 0)

,M2,M2_%_Change
2014-02-01,11178.8,6.45
2008-07-01,7775.4,6.38
1997-07-01,3923.9,5.00
1966-11-01,477.3,4.74
2010-12-01,8801.8,3.60


In [22]:
# Real Gross Domestic Product, Quarterly frequency (Do notice that this indicator frequency is different)

gdp = fred.get_series('GDPC1', frequency='q')

In [23]:
gdp_df = pd.DataFrame(gdp, columns=['GDP'])

In [24]:
# We need to do some data calculation in order to get the year-over-year change:

gdp_df['GDP_%_Change'] = gdp_df['GDP'].pct_change(periods=4).mul(100).round(2)

In [25]:
gdp_df.sample(5, axis = 0)

,GDP,GDP_%_Change
1977-07-01,6313.559,5.77
1959-04-01,3194.429,9.12
1957-10-01,2985.775,0.36
2022-01-01,19924.088,3.68
2015-01-01,17280.647,3.76


### Merge the Economic Indicators Into One Data Frame

In [27]:
# We are using inner join for the first two data frames

final_df = pd.merge(ff_df, u3_df, left_index=True, right_index=True, how='inner')

In [28]:
# Another inner join

final_df = pd.merge(final_df, cpi_df[['CPI_%_Change']], left_index=True, right_index=True, how='inner')

In [29]:
# Another inner join

final_df = pd.merge(final_df, m2_df[['M2_%_Change']], left_index=True, right_index=True, how='inner')

In [30]:
# We are using left join for the GDP because of the different frequency

final_df = pd.merge(final_df, gdp_df[['GDP_%_Change']], left_index=True, right_index=True, how='left')

In [31]:
final_df.head(100)

,FF,U-3,CPI_%_Change,M2_%_Change,GDP_%_Change
1959-01-01,2.48,6.0,1.29,NaN,7.42
1959-02-01,2.43,5.9,1.05,NaN,NaN
1959-03-01,2.74,5.6,0.35,NaN,NaN
1959-04-01,2.95,5.2,0.14,NaN,9.12
1959-05-01,2.86,5.1,0.35,NaN,NaN
...,...,...,...,...,...
1966-12-01,5.40,3.8,3.36,4.57,NaN
1967-01-01,5.13,3.9,3.20,4.24,2.93
1967-02-01,4.88,3.8,2.87,4.41,NaN
1967-03-01,4.58,3.8,2.55,4.82,NaN


### Clean NaN Cells from the Data Frame

In [32]:
# First, we fill the NaN cells in the GDP_%_Change column because of the different frequency

final_df['GDP_%_Change'] = final_df['GDP_%_Change'].fillna(method='ffill')

In [33]:
final_df.isnull().sum()

FF               0
U-3              0
CPI_%_Change     0
M2_%_Change     12
GDP_%_Change     0
dtype: int64

In [34]:
# Second, we will drop the 12 rows with NaN cells in the M2_%_Change column

final_df = final_df.dropna()

### Export the Data Frame to CSV File

In [35]:
final_df.to_csv("US_Indicators.csv")

### Visualization

#### Please use the link to Tableau web site
#### https://public.tableau.com/app/profile/ben.pinhassi/viz/US_Economic_Indicators/Dashboard1